In [7]:
import psycopg2
import os


In [10]:
def get_csv(region='ANY',game_mode='ANY',elo='ANY',version='ANY'):
    #gets data collected into a csv

    #if using sqlalchemy
    # engine_name = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('DB_PORT')
    )

    os.makedirs('MatchData', exist_ok=True)
    csv_path = f'MatchData/{region}_{game_mode}_{elo}_{version}.csv'

    cursor = conn.cursor()

    # Initialize query parts
    query_sql = "SELECT * FROM match_data"
    params = []
    
    # Add conditions dynamically based on inputs
    if region != 'ANY':
        if params:
            query_sql += " AND"
        else:
            query_sql += " WHERE"
        query_sql += " region = %s"
        params.append(region)
    
    if game_mode != 'ANY':
        if params:
            query_sql += " AND"
        else:
            query_sql += " WHERE"
        query_sql += " game_mode = %s"
        params.append(game_mode)
    
    if elo != 'ANY':
        if params:
            query_sql += " AND"
        else:
            query_sql += " WHERE"
        query_sql += " elo LIKE %s"
        params.append(elo + '%')
    
    if version != 'ANY':
        if params:
            query_sql += " AND"
        else:
            query_sql += " WHERE"
        query_sql += " version LIKE %s"
        params.append(version + '%')

    query = cursor.mogrify(query_sql,(params))
    query = query.decode('utf-8')

    try:
        if os.path.exists(csv_path):
            print("Csv found")
        else:
            with open(csv_path,'w') as f:
                cursor.copy_expert("COPY ({}) TO STDOUT WITH CSV HEADER".format(query),f)
            print("Copy to csv successful")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [11]:
#create code to train on every combination, get csv of entire thing
get_csv()

Copy to csv successful
